# Optymalizacja hiperparametrów 

#### Grid Search 

Na przykładzie modelu *XGBoost* widzimy, że niektóre algorytmy są wysoce parametryzowalne, i prawidłowy dobór hiperparametrów potrafi czasami istotnie podnieść jakość naszych predykcji.  
Poznaliśmy już przeszukiwanie po ustalonej siatce parametrów - `GridSearchCV`. Metoda ta ma jednak dwie dość istotne wady:
* rozmiar siatki bardzo szybko rośnie wraz z ilością parametrów. Już dla czterech parametrów, po 5 wartości każdy, dostajemy $5^4 = 625$ kombinacji. W sytuacji, kiedy model uczy nam się minutę (a to w praktyce bardzo optymisyczny przypadek) musimy czekać 10 godzin na przejście pętli.
* niska gęstość siatki powoduje, że możemy nie wstrzelić się w tą optymalną wartość. Przykład:

 https://cdn-images-1.medium.com/max/1600/1*ZTlQm_WRcrNqL-nLnx6GJA.png


#### Random Search 

Częściowym rozwiązaniem tego problemu jest `RandomizedSearch`, czyli losowe przeszukiwanie przestrzeni (zobrazowane na powyższym przykładzie). 
Zalety takiego rozwiązania:
* wspomniane wyżej dokładniejsze przeszukanie każdego z parametrów
* możliwość zadania dowolnego rozkładu (w praktyce dowolnego zaimplementowanego w scipy) dla zmiennych ciągłych
* szybszy czas działania - dzięki temu że przeszukujemy przestrzeń dokładniej, możemy przez to mocno ograniczyć ilość iteracji.  


Jak podawać wartości parametrów do przetestowania? Możemy to zrobić na dwa sposoby:
* lista wartości - wtedy algorytm po prostu będzie losował jedną z wartości z listy
* ciągły rozkład - losujemy wtedy dowolną wartość z tego rozkładu. w `RandomizedSearchCV` przez rozkład rozumiemy obiekt, który posiada metodę `rvs` zwracającą próbkę z tego rozkładu. Dużo rozkładów znajdziemy w module `scipy.stats.distributions` (https://docs.scipy.org/doc/scipy/reference/stats.html)

In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

from sklearn import datasets
cancer = datasets.load_breast_cancer()

X = cancer.data
y = cancer.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
# RandomizedSearchCV??

In [4]:
# GridSearchCV??

In [5]:
param_grid = {
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150, 200, 400],
    'gamma': [0, 0.5, 1, 2],
    'colsample_bytree': [1, 0.8, 0.5],
    'subsample': [1, 0.8, 0.5],
    'min_child_weight': [1, 5, 10]
}

In [6]:
from scipy.stats.distributions import uniform, randint

In [7]:
# ??uniform

Rozkłady przyjmują 2 parametry: `loc`, `scale`.

`[2, 3, 4, 5] --> uniform(2, 3)`

In [8]:
uniform_23 = uniform(2, 3)

In [9]:
# ??randint

In [10]:
# randint(low, high) losuje od low do high-1 włącznie!
randint_25 = randint(2, 6)

In [11]:
param_grid = {
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150, 200, 400],
    'gamma': [0, 0.5, 1, 2],
    'colsample_bytree': [1, 0.8, 0.5],
    'subsample': [1, 0.8, 0.5],
    'min_child_weight': [1, 5, 10]
}

param_distribution = {
    'max_depth': randint(3, 11),
    'learning_rate': uniform(0.001, 0.1-0.001),
    'n_estimators': randint(50, 400),
    'gamma': uniform(0,2),
    'colsample_bytree': uniform(0.5, 0.5),
    'subsample': uniform(0.5, 0.5),
    'min_child_weight': randint(1, 11)
}

# Zad.

Zbudować 
* `RandomSearchCV` 
* `GridSearchCV`
(z wybraną przez siebie ilością iteracji) dla `XGBClassifier` 

Prównaj wyniki.

In [12]:
grid = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)
grid.best_score_

rand = RandomizedSearchCV(XGBClassifier(), param_distribution, cv=5, scoring='f1_macro')
rand.fit(X_train, y_train)
rand.best_score_

KeyboardInterrupt: 